In [ ]:
# pip install mysql


### Conexiń a la BD

In [47]:
import pandas as pd
import pymysql

tweetSchema= {'fechaCreacion': {'format': '%Y-%m-%d %H:%M:%S'}}

myconnection = pymysql.connect(host = 'localhost', user = 'root',password = 'root',db = 'dbtwitter', charset='utf8')

data_db = None
try:
    query = """SELECT * FROM tweet"""
    data_db = pd.read_sql(query, con = myconnection, parse_dates = tweetSchema)
finally:
    myconnection.close()

data_db.head()

,id,idUsuario,fechaCreacion,texto
0,1062887543329325056,916903730066444288,2018-11-15 01:58:32,RT @RedesKeiko: Bienvenidos.\nSomos el Equipo ...
1,1062887563550224389,768444698,2018-11-15 01:58:37,RT @RedesKeiko: Bienvenidos.\nSomos el Equipo ...
2,1062887584937009154,17697214,2018-11-15 01:58:42,RT @adrianaglch: @DocHinostroza La abogada de ...
3,1062887588455964673,1976987462,2018-11-15 01:58:43,RT @IuliusAlfonsus: Por la violación y muerte ...
4,1062887592138588160,1015444443804160000,2018-11-15 01:58:44,RT @RecuerdosPeru: 18/03/2018 Juez Richard Con...


In [48]:
df_tweet_texto = data_db["texto"]
# df_tweet_texto

### Limpieza de Tweets

In [49]:
import string
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [50]:
# Declaramos diccionarios de palabras y simbolos a excluir

stop_words = stopwords.words('spanish')
stop_words.extend(["rt", "aun", "oe"])
exclude = string.punctuation
exclude = exclude + "¿"

lemma = SnowballStemmer('spanish')

# print(stop_words)
# print(exclude)

In [51]:
import re

##def remove_number(list_text):
##    return ''.join([i for i in text if not i.isdigit()])

def remove_three_dots(list_text):
    return [re.sub(r"[a-zA-Z]+(\……|\…)$", " ", texto) for texto in list_text]

def remove_url(list_text):
    return [re.sub(r"http\S+", "", texto).strip() for texto in list_text]

def remove_breakline(list_text):
    return [re.sub('\s+', ' ', texto) for texto in list_text]

def remove_single_quotes(list_text):
    return [re.sub("\'", "", texto) for texto in list_text]

In [52]:
# data como lista de tweets

data = data_db['texto'].values.tolist()
# print(data)

In [53]:
import gensim

txt_free_url = remove_url(data)
txt_free_threedot = remove_three_dots(txt_free_url)
txt_free_breakline = remove_breakline(txt_free_threedot)
txt_free_singlequotes = remove_single_quotes(txt_free_breakline)

def texto_a_palabras(texto:str):
    for sentencia in texto:
        yield(gensim.utils.simple_preprocess(str(sentencia), deacc=True))
        
data_palabras = list(texto_a_palabras(txt_free_singlequotes))

In [55]:
# Palabras limpias

for i in range(len(data_palabras)):
    print(data_palabras[i],"\n")
    True

['rt', 'redeskeiko', 'bienvenidos', 'somos', 'el', 'equipo', 'de', 'redes', 'de', 'keikofujimori', 'desde', 'hoy', 'mientras', 'keiko', 'siga', 'secuestrada', 'por', 'una', 'irracional'] 

['rt', 'redeskeiko', 'bienvenidos', 'somos', 'el', 'equipo', 'de', 'redes', 'de', 'keikofujimori', 'desde', 'hoy', 'mientras', 'keiko', 'siga', 'secuestrada', 'por', 'una', 'irracional'] 

['rt', 'adrianaglch', 'dochinostroza', 'la', 'abogada', 'de', 'keikofujimori', 'se', 'las', 'puso', 'de', 'taquito', 'al', 'pedirle', 'al', 'juez', 'concepcion', 'que', 'se', 'debata', 'cada', 'caso'] 

['rt', 'iuliusalfonsus', 'por', 'la', 'violacion', 'muerte', 'de', 'la', 'nina', 'el', 'fiscal', 'pide', 'meses', 'de', 'prision', 'para', 'el', 'violador', 'asesino', 'confeso', 'para'] 

['rt', 'recuerdosperu', 'juez', 'richard', 'concepcion', 'dicto', 'impedimento', 'de', 'salida', 'para', 'jaime', 'yoshiyama', 'juez', 'cesar', 'sahuanay'] 

['redeskeiko', 'keikofujimori', 'seguimos', 'en', 'la', 'lucha'] 

['rt'

### Palabras Frecuente relacionadas a la marca o nombre

In [56]:
bigram = gensim.models.Phrases(data_palabras, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_palabras], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/home/jkn/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [57]:
print(bigram_mod[data_palabras[2]])

['rt', 'adrianaglch', 'dochinostroza', 'la', 'abogada', 'de', 'keikofujimori', 'se', 'las', 'puso', 'de', 'taquito', 'al', 'pedirle', 'al', 'juez', 'concepcion', 'que', 'se', 'debata', 'cada', 'caso']


In [58]:
import spacy
# python -m spacy download es

def remove_stopwords(texts):
    return [[word for word in gensim.utils.simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words_nostops = remove_stopwords(data_palabras)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('es', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized)

[['redeskeiko', 'bienvenido', 'equipar', 'redar', 'keikofujimori', 'hoy', 'seguir', 'secuestrar', 'irracional'], ['redeskeiko', 'bienvenido', 'equipar', 'redar', 'keikofujimori', 'hoy', 'seguir', 'secuestrar', 'irracional'], ['adrianaglch', 'keikofujimori', 'poner', 'taquito', 'pedirle', 'juez', 'debatir', 'casar'], ['muerte', 'nina', 'fiscal', 'pedir', 'mesar', 'prision', 'violador', 'asesinar', 'confeso'], ['recuerdosperu', 'juez', 'richard', 'dictar', 'impedimento', 'salir', 'jaime', 'yoshiyama', 'juez', 'cesar'], ['redeskeiko', 'keikofujimori', 'seguir', 'luchar'], ['cluzcar'], ['italogiudice', 'rosamariabartra', 'cibernautasfp', 'keikofujimori', 'senora', 'bartra', 'repetir', 'afirmación', 'mediar'], ['redeskeiko', 'bienvenido', 'equipar', 'redar', 'keikofujimori', 'hoy', 'seguir', 'secuestrar', 'irracional'], ['redeskeiko', 'bienvenido', 'equipar', 'redar', 'keikofujimori', 'hoy', 'seguir', 'secuestrar', 'irracional'], ['sullama', 'seguir', 'persecucion', 'politica', 'domingo', '

In [60]:
from gensim import corpora
 
diccionario = corpora.Dictionary(data_lemmatized)
#Corpus
texto = data_lemmatized

In [61]:
corpus = [diccionario.doc2bow(doc) for doc in texto]

print(data_words_bigrams[2])
print(corpus[2])

['adrianaglch', 'dochinostroza', 'abogada', 'keikofujimori', 'puso', 'taquito', 'pedirle', 'juez', 'concepcion', 'debata', 'cada', 'caso']
[(4, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [62]:
print (data_words_bigrams[1])
print (corpus[1])

['redeskeiko', 'bienvenidos', 'equipo', 'redes', 'keikofujimori', 'hoy', 'mientras', 'keiko', 'siga', 'secuestrada', 'irracional']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


In [63]:
[[(diccionario[id], freq) for id, freq in cp] for cp in corpus[0:3]]

[[('bienvenido', 1),
  ('equipar', 1),
  ('hoy', 1),
  ('irracional', 1),
  ('keikofujimori', 1),
  ('redar', 1),
  ('redeskeiko', 1),
  ('secuestrar', 1),
  ('seguir', 1)],
 [('bienvenido', 1),
  ('equipar', 1),
  ('hoy', 1),
  ('irracional', 1),
  ('keikofujimori', 1),
  ('redar', 1),
  ('redeskeiko', 1),
  ('secuestrar', 1),
  ('seguir', 1)],
 [('keikofujimori', 1),
  ('adrianaglch', 1),
  ('casar', 1),
  ('debatir', 1),
  ('juez', 1),
  ('pedirle', 1),
  ('poner', 1),
  ('taquito', 1)]]

In [64]:
print(len(list(diccionario.values())))

140


In [82]:
#Correr y entrenar el modelo LDA sobre la matriz de términos.
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=10,
                                   id2word = diccionario,
                                   random_state=100,
                                   passes=10,
                                   update_every=1,
                                   chunksize=100,
                                   alpha='auto',
                                   per_word_topics=True)

In [83]:
for idx, topic in lda_model.print_topics():
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.007*"keikofujimori" + 0.007*"redeskeiko" + 0.007*"secuestrar" + 0.007*"seguir" + 0.007*"hoy" + 0.007*"redar" + 0.007*"irracional" + 0.007*"equipar" + 0.007*"cluzcar" + 0.007*"bienvenido"

Topic: 1 Word: 0.100*"keikofujimori" + 0.052*"redeskeiko" + 0.048*"juez" + 0.032*"recuerdosperu" + 0.017*"equipar" + 0.017*"salir" + 0.017*"impedimento" + 0.017*"taquito" + 0.017*"jaime" + 0.017*"yoshiyama"

Topic: 2 Word: 0.053*"keikofujimori" + 0.051*"fiscaliaperu" + 0.051*"chilcanodepsico" + 0.051*"pfuerzapopular" + 0.051*"quintopoderpe" + 0.051*"danielurresti" + 0.051*"waykaperu" + 0.007*"bienvenido" + 0.007*"seguir" + 0.007*"equipar"

Topic: 3 Word: 0.150*"keikofujimori" + 0.077*"redeskeiko" + 0.051*"voycontando" + 0.027*"persecucion" + 0.027*"judicial" + 0.027*"destruir" + 0.027*"procesar" + 0.027*"ocurrir" + 0.027*"deber" + 0.027*"decision"

Topic: 4 Word: 0.052*"okno" + 0.052*"corrupto" + 0.052*"jeanpaulcrs" + 0.052*"mas" + 0.052*"tranquilo" + 0.052*"robar" + 0.052*"persecucio

In [84]:
# pip install pyLDAvis

import warnings
import pyLDAvis.gensim

warnings.filterwarnings('ignore')

pyLDAvis.enable_notebook()

vim = pyLDAvis.gensim.prepare(lda_model,corpus,diccionario)


In [79]:
vim[1]

,Category,Freq,Term,Total,loglift,logprob
term,,,,,,
4,Default,29.000000,keikofujimori,29.000000,30.0000,30.0000
6,Default,24.000000,redeskeiko,24.000000,29.0000,29.0000
8,Default,18.000000,seguir,18.000000,28.0000,28.0000
7,Default,18.000000,secuestrar,18.000000,27.0000,27.0000
3,Default,16.000000,irracional,16.000000,26.0000,26.0000
5,Default,16.000000,redar,16.000000,25.0000,25.0000
1,Default,16.000000,equipar,16.000000,24.0000,24.0000
2,Default,16.000000,hoy,16.000000,23.0000,23.0000
0,Default,16.000000,bienvenido,16.000000,22.0000,22.0000


In [11]:
from gensim.test.utils import datapath
from gensim.models.word2vec import Text8Corpus
from gensim.models.phrases import Phrases, Phraser



In [20]:
sentences = Text8Corpus(datapath('testcorpus.txt'))
phrases = Phrases(sentences, min_count=1, threshold=1)



In [22]:
bigram = Phraser(phrases)
bigram

In [33]:

sent = ['trees', 'sun','graph', 'day', 'minors', 'graph', 'trees', 'graph']

In [34]:

print(bigram[sent])

['trees', 'sun', 'graph', 'day', 'minors', 'graph', 'trees_graph']
